In [13]:
import pandas, pyprojroot, warnings

#set up files, prompting user for input (file must be in project folder under data > inputs)
print('What is the file you would like to match? Please include the extension (.csv)')
filename = input()
file = pyprojroot.here('./data/inputs/' + filename)
zip_ej = pyprojroot.here('./data/outputs/ZCTA_EJ.csv')

#pull zip codes and create array of proportions of EJ communities in those zip codes
zipcode_col = "Zip Code" #if your file has a different column title for its zip codes, replace this variable.
df = pandas.read_csv(file)
df = df.dropna(axis='columns', how='all')
lookup = pandas.read_csv(zip_ej)
zips = df[zipcode_col]
ej_prob = [0] * len(zips)
for i in range(0, len(zips)):
    z = int(str(zips[i])[0:5]) #only taking the first 5 characters clears up any zips that are entered as zip+4
    ej_prob[i] = lookup.at[i, 'percent_ej']

#append the array and create the output file
df.insert(len(df.columns), 'Percent EJ', ej_prob)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    outpath = pyprojroot.here('./data/outputs/' + filename[0:-4] + '_EJmatched.csv')
df.to_csv(outpath)
print('Matching complete!')

What is the file you would like to match? Please include the extension (.csv)
EPA-HQ-OAR-2013-0479-8769_attachment_2.csv
Matching complete!
